In [50]:
import pandas as pd
from datetime import datetime
import re

In [51]:
def open_file(filename):
    f = open(filename, "r")
    text = f.read()
    lines = text.split('\n')
    return lines

def open_file(filename):
    f = open(filename, "r")
    text = f.read()
    lines = text.split('\n')
    return lines


def clean_name(name: str) -> str:
    char_add_ons = ['V.O.', "'S COM VOICE",
                    'Cont\'d', 'O.S.', "'S", "VOICE", "INTERCOM"]
    for txt in char_add_ons:
        if txt in name:
            name = name.replace(txt, '')
    return re.sub("[\(\[].*?[\)\]]", "", name.strip())


def get_title(lines):
    series_ind = 0
    for i in range(len(lines)):
        if "STAR TREK: THE NEXT GENERATION" in lines[i]:
            series_ind = i
            break
    return str(lines[series_ind+2]).strip().replace('"','')


def get_date(lines):
    draft_ind = 0
    for i in range(len(lines)):
        if "FINAL DRAFT" in lines[i]:
            series_ind = i
            break
    try:
        date = str(lines[series_ind+2])
        date = date.strip()
        final = datetime.strptime(date, '%B %d, %Y')
    except:
        print(lines[series_ind+2])
        final = "AHHH"
    return final



In [52]:
def get_quotes(lines):
#    characters = [x for x in lines if '\t\t\t\t\t' in x]
    char_index = [i for i in range(len(lines)) if '\t\t\t\t\t' in lines[i]]
    scenes_index = [j for j in range(len(lines)) if 'INT.' in lines[j] or 'EXT.' in lines[j]]

    data = []

    for i in char_index:
        name = lines[i].replace('\t\t\t\t\t', '')
        if "FADE OUT" in name:
            continue

        j = i+1
        next_line = lines[j]

        quote = ""

        while next_line != '':
            next_line = next_line.replace('\t', ' ')
            quote += re.sub("[\(\[].*?[\)\]]", "", next_line)
            j = j+1
            next_line = lines[j]

        scene = lines[max([x for x in scenes_index if x < i])]

        data.append([name, quote,scene])

    test_df = pd.DataFrame(data, columns=['character', 'quote','scene'])
    return (test_df)

def clean_location(loc:str):
    views = ['INT.','EXT.']
    output = loc
    for v in views:
        if v in output:
            output = output[output.index(v)+5:]

    other = ['(','-']
    for c in other:
        if c in output:
            output = output[:output.index(c)-1]
    return(output)

def get_view(loc:str):
    output = None
    views = ['INT.','EXT.']
    for v in views:
        if v in loc:
            output=v
    return(output)


def create_df(filepath: str) -> pd.DataFrame:
    lines = open_file(filepath)
    test_df = get_quotes(lines)
    test_df['location']=test_df['scene'].apply(clean_location)
    test_df['view']=test_df['scene'].apply(get_view)
    test_df['character'] = test_df['character'].apply(clean_name)
    test_df['episode'] = get_title(lines)
    return test_df


In [53]:
create_df('269.txt')

,character,quote,scene,location,view,episode
0,TROI,That's very nice.,1 INT. CLASSROOM,CLASSROOM,INT.,Masks
1,ERIC,"No, it isn't. I just can't get it rig...",1 INT. CLASSROOM,CLASSROOM,INT.,Masks
2,TROI,Not at all... birds have wings of all siz...,1 INT. CLASSROOM,CLASSROOM,INT.,Masks
3,ERIC,But Mrs. Narsu said we should try to make...,1 INT. CLASSROOM,CLASSROOM,INT.,Masks
4,TROI,She's right... it is important for you to...,1 INT. CLASSROOM,CLASSROOM,INT.,Masks
...,...,...,...,...,...,...
421,DATA,I suspect the number was much greater.,59 INT. READY ROOM,READY ROOM,INT.,Masks
422,DATA,My impression is that there were thousand...,59 INT. READY ROOM,READY ROOM,INT.,Masks
423,PICARD,"Well, Data, you may never become fully hu...",59 INT. READY ROOM,READY ROOM,INT.,Masks
424,PICARD,Nonetheless... it's time to stand wat...,59 INT. READY ROOM,READY ROOM,INT.,Masks


In [29]:
lines = open_file('269.txt')
x=get_quotes(lines)
t = x['scene'].unique()

array(['1    INT. CLASSROOM',
       '3    EXT. SPACE - THE ENTERPRISE (OPTICAL)', '4    INT. BRIDGE',
       "5    INT. TROI'S QUARTERS", '8    INT. TEN FORWARD',
       '9    INT. CLASSROOM', '12   INT. ENGINEERING (OPTICAL)',
       '13   INT. BRIDGE', '15   INT. BRIDGE',
       '17   INT. BRIDGE - INCLUDE VIEWSCREEN (OPTICAL)',
       '19   INT. READY ROOM', '22   INT. ENGINEERING',
       '23   INT. ENGINEERING', '25   INT. OBSERVATION LOUNGE',
       "26   INT. DATA'S QUARTERS", '27   INT. BRIDGE',
       "28   INT. DATA'S QUARTERS", '32   INT. TEN FORWARD',
       '33   INT. ENGINEERING', '37   INT. BRIDGE',
       "40   INT. DATA'S QUARTERS", '42   INT. BRIDGE',
       '44   INT. CORRIDOR', '45   INT. BRIDGE - AFT SCIENCE',
       '47   INT. TEMPLE - CONTINUOUS', '53   INT. CORRIDOR',
       '54   INT. TEMPLE', '55   INT. BRIDGE - AFT SCIENCE (OPTICAL)',
       '56   INT. TEMPLE (OPTICAL)', '57   INT. CORRIDOR - CONTINUOUS',
       '58   EXT. SPACE - THE ENTERPRISE (OPTICAL)',


In [28]:
# INT & EXT might hint at scene